In [1]:
from typing import Dict, List, Set, Tuple
import copy

In [2]:
EPSILON = 'EPSILON'

Напоминалка для реализации в каждом классе.
- accept(self, input_string) - функция, которая проверяет, принимает ли автомат входную строку input_string.
- num_states(self) - позволяет получить общее количество состояний детерминированного автомата
- keys(self)-  возвращает список всех ключей в таблице переходов автомата.

In [3]:
class Automat:
    def accept(self, input_string):
        raise NotImplementedError()

    def num_states(self):
        raise NotImplementedError()

    def keys(self):
        raise NotImplementedError()

Реализация недетерминированного автомата. Создание, модификация и работа с недетерминированными автоматами

In [4]:
class NonDeterministicAutomat(Automat):
    def __init__(self, table: Dict[str, List[List[int]]], result_states):
        self.table = table
        self.result_states = result_states
        self.states = None

    #Получение следующего состояния автомата при переходе из заданного состояния по символу
    def nextState(self, state, char):
      if char not in self.table:
        raise ValueError("Символ не может быть принят")
      return self.table[char][state] if char in self.table else None

    #Установка перехода из одного состояния в другое по заданному символу
    def set_trans(self, start, char, finish):
        if char not in self.table:
            self.table[char] = [[] for _ in range(self.num_states())]
        self.table[char][start].append(finish)

    #Выполнение перехода из набора состояний по заданному символу
    def step(self, old_state, char):
      let_step = set()
      for state in old_state:
        let_step.update(self.nextState(state, char))
        if EPSILON in self.table.keys():
            let_step.update(sum([self.e_closure(s) for s in let_step], []))
      return list(let_step) if let_step else []


    def copy(self):
      new_table = copy.deepcopy(self.table)
      result = copy.deepcopy(self.result_states)
      copied_instance = NonDeterministicAutomat(new_table, result)
      return copied_instance

    #Вычисление эпсилон-замыкания
    def e_closure(self, state):
        if EPSILON not in self.table.keys():
            return [state]
        visited = []
        active = [state]
        while len(active) != 0:
            new_active = []
            for s in active:
                new_active.extend(self.table[EPSILON][s])
            visited = list(set(visited + active))
            active = list(set(new_active).difference(visited))
        return visited


    def accept(self, input_string):
        self.states = self.e_closure(0)
        try:
            for c in input_string:
                self.states = self.step(self.states, c)
            for state in self.states:
                if set(self.e_closure(state)).intersection(self.result_states):
                    return True
            return False
        except ValueError:
            return False


    def num_states(self):
        return len(list(self.table.values())[0])


    def keys(self):
        return list(self.table.keys())


Реализация детерминированного автомата. Отправка части функциональности недетерминированному автомату через прокси

In [5]:
class DeterministicAutomat(Automat):
    def __init__(self, table: Dict[str, List[int]], result_states: List[int]):
        self.table = table
        self.result_states = result_states
        proxy_table = {}
        for char, states in table.items():
            proxy_table[char] = [[state] if state is not None else [] for state in states]
        self.proxy = NonDeterministicAutomat(proxy_table, result_states)


    def accept(self, input_string):
        return self.proxy.accept(input_string)


    def num_states(self):
        return self.proxy.num_states()


    def keys(self):
        return self.proxy.keys()


Объединяем два недетерминированных автомата automat1 и automat2 в один автомат путем объединения их таблиц переходов и списков конечных состояний

In [6]:
def combine_automat(automat1, automat2):
    #Объединяю ключи таблиц переходов
    keys = set(list(automat1.table.keys()) + list(automat2.table.keys()))
    #список состояний-результатов для объединенного автомата
    result = [state + automat1.num_states() for state in automat2.result_states]
    result.extend(automat1.result_states)
    new_table = {}
    '''
    Для каждого ключа нового множества: если ключ есть в таблице перехода автомата1,добавляем его переходы в новый список,
      иначе- пустые переходы для каждого состояния автомата1.
      Если ключ есть в таблице переходов автомата2, добавляем его переходы в новый список,
      прибавляя значение состояний автомата 1 к переходам, иначе- добавляем пустые переходы для каждого состояния автомата2
    '''
    for k in keys:
        new_row = []
        if k in automat1.table:
            new_row.extend(automat1.table[k])
        else:
            new_row.extend([[] for _ in range(automat1.num_states())])
        if k in automat2.table:
            new_row.extend([[s + automat1.num_states() for s in states] for states in automat2.table[k]])
        else:
            new_row.extend([[] for _ in range(automat2.num_states())])
        new_table[k] = new_row
    table = new_table
    result_states = result
    merged_automat = NonDeterministicAutomat(table, result_states)
    return merged_automat

Задаем ход работы с автоматами для использования операторов

In [7]:
#для ,
def matching(automat1, automat2):
    merged = combine_automat(automat1, automat2)
    for start in automat1.result_states:
        merged.set_trans(start, EPSILON, automat1.num_states())
    merged.result_states = [s + automat1.num_states() for s in automat2.result_states]
    return merged

In [8]:
#для ;
def next(automat1, automat2):
    merged = combine_automat(automat1, automat2)
    shifted_finals = [f + 1 for f in merged.result_states]
    shifted_table = {}
    for char, state_list in merged.table.items():
        shifted_table[char] = [[]] + [[state + 1 for state in states] for states in state_list] + [[]]
    new = NonDeterministicAutomat(table=shifted_table, result_states=[])
    new.set_trans(0, EPSILON, 1)
    new.set_trans(0, EPSILON, automat1.num_states() + 1)
    for f in shifted_finals:
        new.set_trans(f, EPSILON, new.num_states() - 1)
    new.result_states = [new.num_states() - 1]
    result_next = new
    return result_next

In [9]:
#для *
def multiplication(automat1):
    shifted_finals = [f + 1 for f in automat1.result_states]
    shifted_table = {}
    for char, state_list in automat1.table.items():
        shifted_table[char] = [[]] + [[state + 1 for state in states] for states in state_list] + [[]]
    new = NonDeterministicAutomat(table=shifted_table, result_states=[])
    for f in shifted_finals:
        new.set_trans(f, EPSILON, 1)
        new.set_trans(f, EPSILON, new.num_states() - 1)
    new.set_trans(0, EPSILON, 1)
    new.set_trans(0, EPSILON, new.num_states() - 1)
    new.result_states = [new.num_states() - 1]
    result_mult = new
    return result_mult

In [10]:
# для +
def merged(automat1):
    return matching(automat1, multiplication(automat1))

In [11]:
#для #
def generalized_iteration(automat1, automat2):
    return matching(automat1, multiplication(matching(automat2, automat1)))

In [12]:
# возможность принять пустую строку
def optional(automat1):
    automat1_copy = automat1.copy()
    for f in automat1_copy.result_states:
        automat1_copy.set_trans(0, EPSILON, f)
    return automat1_copy

создание недетерминированного автомата для actual_string

In [13]:
def primitive_fnda(actual_string):
    table: Dict[str, List[List[int]]] = {}
    for i, c in enumerate(actual_string):
        if c not in table:
            table[c] = [[] for _ in range(len(actual_string) + 1)]
        table[c][i].append(i + 1)
    table=table
    result_states=[len(actual_string)]
    return NonDeterministicAutomat(table,result_states)


operations = {
    '*': multiplication,
    '+': merged,
    ';': next,
    ',': matching,
    '#': generalized_iteration,
    '?': optional
    }


priorities = {
    ';': 0,
    '#': 1,
    ',': 1,
    '*': 2,
    '+': 2,
    '?': 2
    }

binary = [';', '#', ',']
unary = ['*', '+', '?']


def is_character(c):
    return c not in {*operations.keys(), '(', ')'}




преобразование регулярки

In [14]:
def regex_p(regexp):
    if len(regexp) == 0:
        return ''
    new = []
    last = None
    for c in regexp:
        if last is None:
            last = c
            new.append(c)
            continue
        if last in unary and c == '(' \
                or last in unary and is_character(c) \
                or is_character(last) and is_character(c) \
                or last == ')' and is_character(c) \
                or is_character(last) and c == '(':
            new.append(',')
        new.append(c)
        last = c
    return ''.join(new)

строим недетерминированный конечный автомат на основе регулярки

In [15]:
def construct_fnda(regexp):
    data = ''
    op_stack = []
    eval_stack = []

    # Функция для обработки операторов в соответствии с их приоритетами. Извлекаем операторы из стека операторов
    def process_operator(priority=-1):
        while len(op_stack) != 0 \
                and op_stack[-1] != '(' \
                and (op_stack[-1] not in operations.keys() or priorities[op_stack[-1]] > priority):
            op = op_stack[-1]
            if op in binary:
                eval_stack.append(operations[op](eval_stack[-2], eval_stack[-1]))
                eval_stack.pop(-2)
                eval_stack.pop(-2)
                op_stack.pop()
            elif op in unary:
                eval_stack.append(operations[op](eval_stack[-1]))
                eval_stack.pop(-2)
                op_stack.pop()
        if priority == -1 and len(op_stack) != 0 and op_stack[-1] == '(':
            op_stack.pop()
    regexp = regex_p(regexp)

    for c in regexp:
        if c in list(operations.keys()) + ['(', ')']:
            if data != '':
                eval_stack.append(primitive_fnda(data))
            data = ''
        if c in operations:
            if len(op_stack) == 0 or op_stack[-1] in ['(', ')'] or priorities[op_stack[-1]] < priorities[c]:
                op_stack.append(c)
            else:
                process_operator(priorities[c])
                op_stack.append(c)
        elif c == '(':
            op_stack.append('(')
        elif c == ')':
            process_operator()
        else:
            data += c

    if data != '':
        eval_stack.append(primitive_fnda(data))
    process_operator()
    return eval_stack[-1]



конвертация недерменированного конечного автомата в детерменированный.

In [16]:
def convert_to_fda(fnda):
    links = []
    # Множество новых состояний, начиная с эпсилон состояния 0
    newStates = [set(fnda.e_closure(0))]
    visitedStates = []
    keys = [x for x in list(fnda.table.keys()) if x != EPSILON]
    while len(newStates) > 0:
        tmp = newStates.pop()
        # Если состояние уже посещено, пропускаем его
        if tmp in visitedStates:
            continue
        # Добавляем состояние в список посещенных
        visitedStates.append(tmp)
        # Получаем новое состояние после перехода по символу
        for char in keys:
            newTmp = set(fnda.step(tmp, char))
            if len(newTmp) != 0:
                newStates.append(newTmp)
                links.append((tmp, char, newTmp))
    formatted_links = []
    # Преобразование ссылок в индексы состояний
    for link in links:
        formatted_links.append((visitedStates.index(link[0]), link[1], visitedStates.index(link[2])))
    links = formatted_links
    old_final = set(fnda.result_states)
    # Индексы финальных состояний в полученном DFA
    result = [i for i, s in enumerate(visitedStates) if s.intersection(old_final)]
    new_table = {}
    for k in keys:
        new_table[k] = [None for _ in enumerate(visitedStates)]
    # Заполнение таблицы переходов
    for link in links:
        new_table[link[1]][link[0]] = link[2]
    table=new_table
    result_states=result
    return DeterministicAutomat(table, result_states)


минимизация детерминированного конечного автомата

In [17]:
def minimize_fda(fda):
    def split_set(target, splitter, split_char):
        setA = set()
        setB = set()
        for condition in target:
            if fda.table[split_char][condition] in splitter:
                setA.add(condition)
            else:
                setB.add(condition)
        return setA, setB

    # Инициализация списка классов эквивалентности
    sets = [{*fda.result_states}]
    non_final = {*list(range(fda.num_states()))}.difference(fda.result_states)
    if len(non_final) > 0:
        sets.append(non_final)
    queue = []
    # Формирование очереди разбиений классов эквивалентности
    for symbol in fda.keys():
        for classs in sets:
            queue.append((classs, symbol))
    # Построение классов эквивалентности
    while len(queue) > 0:
        splitter, char = queue.pop(0)
        for classs in sets:
            setA, setB = split_set(classs, splitter, char)
            if len(setA) > 0 and len(setB) > 0:
                sets.remove(classs)
                sets.extend([setA, setB])
                if (classs, char) in queue:
                    queue.remove((classs, char))
                    queue.append((setA, char))
                    queue.append((setB, char))
                else:
                    if len(setA) < len(setB):
                        queue.append((setA, char))
                    else:
                        queue.append((setB, char))

    # Перестановка классов эквивалентности так, чтобы первым был класс состояния 0
    first_state_index = [sets.index(classs) for classs in sets if 0 in classs][0]
    first_state = sets.pop(first_state_index)
    sets.insert(0, first_state)

    states = len(sets)
    new_table = {k: [None] * states for k in fda.keys()}
    # Формирование новой таблицы переходов
    for i, classs in enumerate(sets):
        for condition in classs:
            for symbol in fda.keys():
                new_indexes = [sets.index(classs) for classs in sets if fda.table[symbol][condition] in classs]
                new_table[symbol][i] = None if len(new_indexes) == 0 else new_indexes[0]

    # Формирование нового множества финальных состояний
    result = [sets.index(classs) for classs in sets if classs.intersection(fda.result_states)]
    table=new_table
    result_states=result
    return DeterministicAutomat(table, result_states)

Функция для проверки

In [18]:
def match_string(regex, input_string):
    fnda = construct_fnda(regex)
    fda = convert_to_fda(fnda)
    minimized_fda = minimize_fda(fda)
    current_state = 0
    for char in input_string:
        if char not in minimized_fda.keys():
            return False
        nextState = minimized_fda.table[char][current_state]
        if nextState is None:
            return False
        current_state = nextState
    return current_state in minimized_fda.result_states


Возвращает результат проверки, True если входная строка соответствует регулярному выражению, и False в противном случае.

In [19]:
regex = input("Введите регулярное выражение: ")
input_string = input("Введите строку для проверки: ")

res = match_string(regex, input_string)
print("Результат:", res)


Введите регулярное выражение: a*b+c
Введите строку для проверки: aabc
Результат: True


In [20]:
regex = input("Введите регулярное выражение: ")
input_string = input("Введите строку для проверки: ")

res = match_string(regex, input_string)
print("Результат:", res)

Введите регулярное выражение: a*b+c
Введите строку для проверки: abbc
Результат: True


In [21]:
regex = input("Введите регулярное выражение: ")
input_string = input("Введите строку для проверки: ")

res = match_string(regex, input_string)
print("Результат:", res)

Введите регулярное выражение: a*b+c
Введите строку для проверки: aaabc
Результат: True


In [22]:
regex = input("Введите регулярное выражение: ")
input_string = input("Введите строку для проверки: ")

res = match_string(regex, input_string)
print("Результат:", res)

Введите регулярное выражение: a*b+c
Введите строку для проверки: abcc
Результат: False


In [25]:
regex = input("Введите регулярное выражение: ")
input_string = input("Введите строку для проверки: ")

res = match_string(regex, input_string)
print("Результат:", res)

Введите регулярное выражение: a*m*k
Введите строку для проверки: amk
Результат: True


In [32]:
regex = input("Введите регулярное выражение: ")
input_string = input("Введите строку для проверки: ")

res = match_string(regex, input_string)
print("Результат:", res)

Введите регулярное выражение: a*m,k
Введите строку для проверки: amk
Результат: True
